In [8]:
import numpy as np
import pandas as pd

# keras import for the dataset
from keras.datasets import mnist

In [9]:
# Read the dataset imdb.txt. The seperator is tab
# Show the first 10 records
data = pd.read_csv('../input/imdb.txt', sep = '\t')
data.head(10)

sentence  opinion
0  A very, very, very slow-moving, aimless movie ...        0
1  Not sure who was more lost - the flat characte...        0
2  Attempting artiness with black & white and cle...        0
3       Very little music or anything to speak of.          0
4  The best scene in the movie was when Gerardo i...        1
5  The rest of the movie lacks art, charm, meanin...        0
6                                Wasted two hours.          0
7  Saw the movie today and thought it was a good ...        1
8                               A bit predictable.          0
9  Loved the casting of Jimmy Buffet as the scien...        1

In [10]:
# How many records are in the dataset?
data.shape

(748, 2)

In [11]:
data.count()

sentence    748
opinion     748
dtype: int64

In [12]:
# Remove stopwords and use stemming for the inputdata
# Show the first 10 records

import nltk

# Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
  
stop_words_en = set(stopwords.words('english')) 
punctuations="?:!.,;<>/\+-"

from nltk.stem.snowball import SnowballStemmer

englishStemmer=SnowballStemmer("english")

def remove_stopwords_and_use_stemming(text):
    words = word_tokenize(text)
    result = ''
    for word in words:
        if word not in stop_words_en and word not in punctuations:
            result += englishStemmer.stem(word) + ' '
    return result

data['sentence'] = data['sentence'].map(remove_stopwords_and_use_stemming)
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svre257\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


sentence  opinion
0  a slow-mov aimless movi distress drift young man         0
1  not sure lost flat charact audienc near half w...        0
2  attempt arti black & white clever camera angl ...        0
3                      veri littl music anyth speak         0
4  the best scene movi gerardo tri find song keep...        1

In [13]:
# Create X (= features) and y (= target)
X = data['sentence']
y = data['opinion']

In [14]:
# Use TfidfVectorizer to transform X
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X = vec.fit_transform(data['sentence'])
print(len(vec.get_feature_names()))

2481


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [15]:
# Split in test and training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

In [16]:
# Use one-hot encoding for y
# Print the dimensions before the one-hot encoding
# Print the dimensions after one-hot encoding

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
print(y_train.shape)
print(y_train[0])  # one sample’s categorical data
y_test = to_categorical(y_test)
print(y_test.shape)

(523, 2)
[0. 1.]
(225, 2)


In [7]:
# keras imports for building our neural network
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation

# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(512, input_shape=(2481,)))
model.add(Activation('sigmoid'))                            
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(2))
model.add(Activation('softmax'))


In [8]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [9]:
# training the model and saving metrics in history
model.fit(X_train, y_train,epochs=20,verbose=2)  

In [10]:
# saving the model
import os
save_dir = "./"
model_name = 'keras_imdb.h5'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

In [11]:
# What is the loss?
# What is the accuracy? 
mnist_model = load_model('keras_imdb.h5')
loss, accuracy = mnist_model.evaluate(X_test, y_test)

print("Test Loss", loss)
print("Test Accuracy", accuracy)

In [12]:
# load the model and create predictions on the test set
model = load_model('keras_imdb.h5')

predictions = model.predict(X_test)

In [ ]:
incorrect_predictions = []

for i, (p, e) in enumerate(zip(predictions, y_test)):
    predicted, expected = np.argmax(p), np.argmax(e)

    if predicted != expected:  # prediction was incorrect
        incorrect_predictions.append((i, predicted, expected))